# Modular table scraper
#### A script to get the table data from any table in the HTML  of a website
###### It also has the capability to show you what tables are in the page and then select the one you want.
######  To do this, make the second criteria None. Else, put the number of the wanted table there.
###### It returns a faithful representation of the table that can then be refined.
###### By being modular and automated, it then allows for the scraping of countless related pages (such as pages in a series) and for this data to then be collated.

In [1]:
def getTable(url, tableNumber):
        
    import pandas as pd
    from bs4 import BeautifulSoup
    
    df = pd.DataFrame()
    
    #########
    
    def getSoup(url):
    
        from requests import get
        from bs4 import BeautifulSoup
        import requests

        user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14'
        headers = {'User-Agent': user_agent,'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}
        response = get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'lxml')

        return soup

    #######

    def getTables(soup, print_on):

        import pandas as pd
        from bs4 import BeautifulSoup

        tables = soup.find_all('table')

        if print_on == True:
            for table in tables:
                attrs = list(table.attrs)
                print(f'\n--- {tables.index(table)}/{len(tables)} --- {attrs}')
                for attr in ['summary', 'title']:
                    if attr in attrs:
                        print(attr, ' ' * (10 - len(str(attr))), table[attr])

        return tables

    ######

    def getTableHead(table):

        import pandas as pd
        import bs4
        from bs4 import BeautifulSoup

        df = pd.DataFrame()

        thead = table.find('thead')

        if thead != None:
            thead_trs = thead.find_all('tr')

            if len(thead_trs) > 0:       
                for tr in thead_trs:

                    n = thead_trs.index(tr)

                    row_vals = [x for x in tr if x != '\n' and type(x) != bs4.element.Comment]
                    row_vals = [x.text.strip() if type(x) == bs4.element.Tag else x for x in row_vals]
                    #print(row_vals)

                    for val in row_vals:                        
                        df.loc[n, row_vals.index(val)] = val

        return df

    ######

    def getTableBody(table):

        import pandas as pd
        from bs4 import BeautifulSoup

        columns = None
        df = pd.DataFrame(columns=columns)   

        tbody = table.find('tbody')
        if tbody==None:
            tbody = table

        rows = tbody.find_all('tr')

        for row in rows:

            nr = df.shape[0]

            headers = [x.text.strip() for x in row.find_all('th')]
            data = [x.text.strip() for x in row.find_all('td')]
            full_row = headers + data

            if df.shape[1] != len(full_row):  
                if len(row.text.strip()) == 0:    ### blank rows
                    full_row = ['' for x in range(0,df.shape[1])] 

            row_cols = [x for x in range(0, len(full_row))]
            df.loc[nr, row_cols] = full_row


        table_title = table['title'] if 'title' in table.attrs else table['summary'] if 'summary' in table.attrs else None

        return df.values, table_title

    #############################
        
    soup = getSoup(url)

    displayTables = True if tableNumber == None else False
    tables = getTables(soup, displayTables)
    
    print(len(tables))
    
    if len(tables) == 0:
        cols, values, table_title = None, None, None
        print('!! NO TABLES FOUND IN PAGE !!')
        
    else:
        
        if tableNumber == None:
            
            if len(tables) == 1:
                tableNumber = 0
                print('One table found in page')
            elif len(tables) > 1:
                tableNumber = int(input('Which table? '))

        table = tables[tableNumber]

        headers = getTableHead(table)
        values, table_title = getTableBody(table)

        if len(headers.values) == 0:  ### no header
            cols = None

        elif len(headers.values) == 1:  # normal header
            cols = headers.values[0]

        elif len(headers.values) > 1:   # multiindex header
            if headers.shape[1] == values.shape[1]:  # ... that follows pattern

                cols = []
                for i in range(0,headers.shape[0]):    
                    cols.append(headers.loc[i].values)

            else:   # that does not

                cols = None
                print('\n\t!!! columns headers must be done manually !!!\n')   

    df = pd.DataFrame(columns=cols, data=values)
    if len(tables) != 0:
        df['table_title'] = table_title

    return df

#### Example

In [5]:
url = 'https://itsitecoreblobvecprd.blob.core.windows.net/public-files/historical-results/state2018/tcpbyvotingcentrealbertparkdistrict.html'
df = getTable(url, 1)
df

2


BOND, Andrew  \
                                   Voting Centres      LIBERAL   
0                                        Bridport          704   
1                                   Kerferd South          715   
2                                     Middle Park          180   
3                                  Port Melbourne          737   
4                             Port Melbourne East          282   
5                             Port Melbourne West          441   
6                                       Sandridge          389   
7                                 South Melbourne          595   
8                                       Southbank          423   
9                                        St Kilda          122   
10                                  St Kilda Park          438   
11                                 St Kilda South          290   
12                           Ordinary Votes Total         5316   
13                                                               
14                                   Postal Votes         1996   
15                                    Early Votes         6116   
16                          Marked As Voted Votes            2   
17                              Provisional Votes          154   
18                                   Absent Votes         1143   
19                                          Total        14727   
20                                                               
21  Percentage of formal vote polled by candidate       36.90%   

            FOLEY, Martin       NaN                                    \
   AUSTRALIAN LABOR PARTY Mis-sorts Informal votes Total votes polled   
0                    1152         0             83               1939   
1                    1080         0             79               1874   
2                     392         1             27                600   
3                    1183         1            116               2037   
4                     603         0             58                943   
5                     823         0             55               1319   
6                     780         0             89               1258   
7                    1168         0             94               1857   
8                     977         0             49               1449   
9                     402         0             35                559   
10                   1334         0             70               1842   
11                    944         0             66               1300   
12                  10838         2            821              16977   
13                                                                      
14                   2279         0            237               4512   
15                   9878         0            696              16690   
16                      2         0              0                  4   
17                    267         0             16                437   
18                   1922         0            169               3234   
19                  25186         2           1939              41854   
20                                                                      
21                 63.10%                                               

                                         table_title  
                                                      
0   Two candidate preferred results by voting centre  
1   Two candidate preferred results by voting centre  
2   Two candidate preferred results by voting centre  
3   Two candidate preferred results by voting centre  
4   Two candidate preferred results by voting centre  
5   Two candidate preferred results by voting centre  
6   Two candidate preferred results by voting centre  
7   Two candidate preferred results by voting centre  
8   Two candidate preferred results by voting centre  
9   Two candidate preferred results by voting centre  
10  Two candidate preferred results by voting centre

In [6]:
url = 'https://itsitecoreblobvecprd.blob.core.windows.net/public-files/historical-results/state2018/tcpbyvotingcentrealbertparkdistrict.html'
df = getTable(url, None)
df


--- 0/2 --- ['title', 'summary']
summary     Elected member
title       Elected member

--- 1/2 --- ['class', 'width', 'cellpadding', 'cellspacing', 'title', 'summary']
summary     Two candidate preferred results by voting centre
title       Two candidate preferred results by voting centre
2


Which table?  0


,0,table_title
0,"FOLEY, MartinAUSTRALIAN LABOR PARTY",Elected member
